# DM 2 Application numérique

In [182]:
import numpy as np
import pandas as pd
from math import *
from scipy import stats
import statsmodels.api as sm 

df=pd.read_csv("Titanic.csv")
l=len(df['PassengerId'])
df['Age'].fillna(df['Age'].mean(),inplace=True)
S=[]
for i in range(l):
    if df['Sex'][i]=='male':
        S.append(1)
    else:
        S.append(0)
new_df=df.assign(Sexe=S)
print(new_df)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex        Age  \
0                              Braund, Mr. Owen Harris    male  22.000000   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.000000   
2                               Heikkinen, Miss. Laina  female  26.000000   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.000000   
4                             Allen, Mr. William Henry    male  35.000000   
..                                                 ...     ...        ...   
886 

#### 1. Déterminer le nombre et la proportion de passagers décédés.

In [17]:
compteur=0
for i in range(l):
    if df['Survived'][i]==0:
        compteur+=1

print("Le nombre de passagers décdés est:")
print(compteur)
print("La proportion de passagers décédés est:")
print(compteur/l)

Le nombre de passagers décdés est:
549
La proportion de passagers décédés est:
0.6161616161616161


#### 2. Déterminer le pourcentage d'hommes et de femmes parmi les personnes décédées et les survivants. Qu'observe-t-on?

In [87]:
compteur_h_déc=0
compteur_h_viv=0
for i in range(l):
    if df['Survived'][i]==0:
        if df['Sex'][i]==1:
            compteur_h_déc+=1
    else:
        if df['Sex'][i]==1:
            compteur_h_viv+=1

print("Il y a "+str((compteur_h_déc/compteur)*100)+" pourcents d'hommes et "+str(100-(compteur_h_déc/compteur)*100)+" pourcents de femmes parmi les personnes décédées")
print("Il y a "+str((compteur_h_viv/(l-compteur))*100)+" pourcents d'hommes et "+str(100-(compteur_h_viv/(l-compteur))*100)+" pourcents de femmes parmi les survivants")


Il y a 85.24590163934425 pourcents d'hommes et 14.754098360655746 pourcents de femmes parmi les personnes décédées
Il y a 31.871345029239766 pourcents d'hommes et 68.12865497076024 pourcents de femmes parmi les survivants


On observe une grande majorité d'hommes parmis les personnes décédées et une grande majorité de femmes parmi les survivants ce qui est en accord avec le fait que les femmes et les enfants ont embarqué en priorité dans les canaux de sauvetage pour survivre.

#### 3. Reprendre cette analyse pour les différentes classes. Qu'observe-t-on?

In [23]:
compteur_classe1_viv=0
compteur_classe2_viv=0
compteur_classe1_déc=0
compteur_classe2_déc=0
for i in range(l):
    if df['Survived'][i]==0:
        if df['Pclass'][i]==1:
            compteur_classe1_déc+=1
        if df['Pclass'][i]==2:
            compteur_classe2_déc+=1
    else:
        if df['Pclass'][i]==1:
            compteur_classe1_viv+=1
        if df['Pclass'][i]==2:
            compteur_classe2_viv+=1

P1d=(compteur_classe1_déc/compteur)*100
P1v=(compteur_classe1_viv/(l-compteur))*100
P2d=(compteur_classe2_déc/compteur)*100
P2v=(compteur_classe2_viv/(l-compteur))*100
P3d=100-P2d-P1d
P3v=100-P2v-P1v

print("Il y a "+str(P1d)+" pourcents de classe 1, "+str(P2d)+" pourcents de classe 2 et "+str(P3d)+" pourcents de classe 3 parmi les personnes décédées.")
print("Il y a "+str(P1v)+" pourcents de classe 1, "+str(P2v)+" pourcents de classe 2 et "+str(P3v)+" pourcents de classe 3 parmi les survivants.")



Il y a 14.571948998178508 pourcents de classe 1, 17.66848816029144 pourcents de classe 2 et 67.75956284153004 pourcents de classe 3 parmi les personnes décédées.
Il y a 39.76608187134503 pourcents de classe 1, 25.438596491228072 pourcents de classe 2 et 34.7953216374269 pourcents de classe 3 parmi les survivants.


On observe que parmi les personnes décédées il y a une majorité de personnes de la 3ème classe et parmi les survivants il y a la même proportion de personnes de classe 1 et de classe 3. 
Si l'on tient compte du fait que beaucoup plus de passagers voyageaient en classe 3, on se rend compte que les passagers des classes les plus chères ont eu beaucoup plus de chance de survivre à la catastrophe: ils ont eu plus facilement accés aux canaux de sauvetage.


#### 4. Calculer la matrice de corrélation des covariables. Qu'observe-t-on?

In [190]:
x = new_df.select_dtypes(np.number).drop(["PassengerId"],axis=1)
matrix=x.corr()
print(matrix)

          Pclass       Age     SibSp     Parch      Fare      Sexe
Pclass  1.000000 -0.331339  0.083081  0.018443 -0.549500  0.131900
Age    -0.331339  1.000000 -0.232625 -0.179191  0.091566  0.084153
SibSp   0.083081 -0.232625  1.000000  0.414838  0.159651 -0.114631
Parch   0.018443 -0.179191  0.414838  1.000000  0.216225 -0.245489
Fare   -0.549500  0.091566  0.159651  0.216225  1.000000 -0.182333
Sexe    0.131900  0.084153 -0.114631 -0.245489 -0.182333  1.000000


On peut remarquer que l'âge est assez corrélé avec la classe des passagers: on peut penser que les personnes les plus jeunes ont eu moins de moyens pour voyager en première classe.

Enfin, le nombre d'enfants/ de parents/ de conjoints à bord n'est pas corrélé avec la variable 'Survived': le sauvetage n'a donc pas eu lieu en regroupant les familles entre elles ce qui peut être surprenant.

#### 5. On inclut d'abord tous les paramètres dans la régression. Calculer les intervalles de confiance à 95% pour l'ensemble des paramètres. Qu'observe-t-on?

In [185]:
new_df["Survived"] = new_df["Survived"].astype('category')
Y = new_df['Survived'].cat.codes
x = new_df.select_dtypes(np.number).drop(["PassengerId"],axis=1)

X = sm.add_constant(x)
model = sm.Logit(Y, X)
result = model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.442607
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  891
Model:                          Logit   Df Residuals:                      884
Method:                           MLE   Df Model:                            6
Date:                Wed, 19 Oct 2022   Pseudo R-squ.:                  0.3353
Time:                        20:50:03   Log-Likelihood:                -394.36
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 7.794e-83
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.9604      0.533      9.308      0.000       3.916       6.005
Pclass        -1.0843      0.

On remarque que les coefficients de Pclass, Age, SibSp, Parch et Sexe sont négatifs ainsi lorsqu'ils augmentent les chances de survie diminuent. Donc on a plus de chance de survivre en étant une jeune femme en première classe sans famille à bord. Néanmoins plus le billet est cher plus les chances de survie sont élevées car le coefficient de Fare est positif. 
Ces influences sont plus ou moins fortes en fonction de la valeur absolue de ces coefficients (le paramètre le plus influent étant le sexe).

#### 6. Déterminer la p-valeur du test: $$H_0:\beta_{Parch}=0$$  $$ H_1:\beta_{Parche}\neq0$$
#### Que peut-on conclure?


On observe grâce au tableau de la question précédente que cette p-valeur vaut 0.340 
Cette valeur est relativement élevée ce qui ne nous permet pas de rejeter l'hypothèse nulle.
Ainsi, le nombre de parents/ d'enfants à bord ne semble pas avoir d'influence sur la survie.

#### 7. On reprend l'analyse en éliminant la variable Parch. Calculer les intervalles de confiance à 95%. Que peut-on en conclure? 

Les intervalles de confiance restent les mêmes, on modifie simplement le vecteur X qui ne contient plus la variable Parch

In [192]:
x2=new_df.select_dtypes(np.number).drop(["PassengerId",'Parch'],axis=1)
X2 = sm.add_constant(x2)
model2 = sm.Logit(Y, X2)
result2 = model2.fit()
print(result2.summary())

Optimization terminated successfully.
         Current function value: 0.443127
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  891
Model:                          Logit   Df Residuals:                      885
Method:                           MLE   Df Model:                            5
Date:                Wed, 19 Oct 2022   Pseudo R-squ.:                  0.3346
Time:                        20:53:56   Log-Likelihood:                -394.83
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 1.312e-83
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.9424      0.531      9.312      0.000       3.902       5.983
Pclass        -1.0982      0.

#### 8. Déterminer la p-valeur du test :$$H_0:\beta_{Fare}=0$$  $$ H_1:\beta_{Fare}\neq0$$

On observe que cette p-valeur vaut 0.292 ce qui est relativement élevé et ce qui ne nous permet donc pas de rejeter l'hypothèse nulle. Ainsi, le prix du billet ne semble pas non plus avoir d'influence sur la survie.


#### 9. On reprend en éliminant la variable Parch et Fare. Calculer les intervalles de confiance à 95%. Que peut-on en conclure?

On garde encore les mêmes intervalles mais on modifie le vecteur X:

In [193]:
x3=new_df.select_dtypes(np.number).drop(["PassengerId",'Parch','Fare'],axis=1)
X3 = sm.add_constant(x3)
model3 = sm.Logit(Y, X3)
result3 = model3.fit()
print(result3.summary())

Optimization terminated successfully.
         Current function value: 0.443796
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                  891
Model:                          Logit   Df Residuals:                      886
Method:                           MLE   Df Model:                            4
Date:                Wed, 19 Oct 2022   Pseudo R-squ.:                  0.3336
Time:                        20:54:57   Log-Likelihood:                -395.42
converged:                       True   LL-Null:                       -593.33
Covariance Type:            nonrobust   LLR p-value:                 2.236e-84
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.1920      0.478     10.854      0.000       4.254       6.130
Pclass        -1.1724      0.

#### 10. Dans ce modèle, quelles sont les probabilités de survie d'un homme dans un cas, et d'une femme dans un autre cas, chacun âgé de 22, sans famille et voyageant en 1ère classe?

On a la formule $$P(Y)=\frac{1}{1+e^{-\left(b_0+b_1x_1+b_2x_2+b_3x_3+b_4x_4\right)}}$$

avec $P(Y)$ la probabilité de survie, les $b_i$ les coefficients des différents $x_i$ qui sont les variables explicatives

In [198]:
def P(x1,x2,x3,x4):
    return 1/(1+exp(-(5.1920+(-1.1724)*x1+(-0.0398)*x2+(-0.3578)*x3+(-2.7398)*x4)))

print("La probabilité pour une femme de 22 ans sans famille et voyageant en première classe de survivre était de:")
print(P(1,22,0,0))
print("Et celle d'un homme était de:")
print(P(1,22,0,1))

La probabilité pour une femme de 22 ans sans famille et voyageant en première classe de survivre était de:
0.9586716530428824
Et celle d'un homme était de:
0.5996963356777223


Cela semble cohérent étant donné qu'on a remarqué dans les questions précédentes qu'être une femme jeune voyageant en première classe et sans famille donnait les meilleures chances de survie.